In [27]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth 
from pyECLAT import ECLAT

apriori

In [13]:
data = pd.read_csv('supermarket_sales - Sheet1.csv')
data.columns
dataset=data['Product_line']

In [17]:
dataset=data.Product_line.unique()

In [35]:
# Transactions done in Health and beauty
basket_handb = (data[data['Product_line'] =="Health and beauty"]
		.groupby(['Invoice_ID', 'City'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('Invoice_ID'))

# Transactions done in the Electronic accessories
basket_Ea = (data[data['Product_line'] =="Electronic accessories"]
		.groupby(['Invoice_ID', 'City'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('Invoice_ID'))

# Transactions done in Home and lifestyle
basket_handl = (data[data['Product_line'] =="Home and lifestyle"]
		.groupby(['Invoice_ID', 'City'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('Invoice_ID'))
#Transactions done in Fashion accessories
basket_Fa = (data[data['Product_line'] =="Fashion accessories"]
		.groupby(['Invoice_ID', 'City'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('Invoice_ID'))

In [38]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
	if(x<= 0):
		return 0
	if(x>= 1):
		return 1

# Encoding the datasets
basket_encoded = basket_handb.applymap(hot_encode)
basket_handb = basket_encoded

basket_encoded = basket_Ea.applymap(hot_encode)
basket_Ea = basket_encoded

basket_encoded = basket_handl.applymap(hot_encode)
basket_handl = basket_encoded

basket_encoded = basket_Fa.applymap(hot_encode)
basket_Fa = basket_encoded

In [63]:
# Building the model
frq_items = apriori(basket_encoded, min_support = 0.0001, use_colnames = True)
frq_items

,support,itemsets
0,0.348315,(Mandalay)
1,0.365169,(Naypyitaw)
2,0.286517,(Yangon)


In [18]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

fpgrowth

In [19]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [20]:
fpgrowth(df, min_support=0.6)

,support,itemsets
0,1.0,(5)
1,0.8,(3)
2,0.6,(10)
3,0.6,(8)
4,0.6,(6)
5,0.8,"(3, 5)"
6,0.6,"(10, 5)"
7,0.6,"(8, 3)"
8,0.6,"(8, 5)"
9,0.6,"(8, 3, 5)"


In [21]:
fpgrowth(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,1.0,(Kidney Beans)
1,0.8,(Eggs)
2,0.6,(Yogurt)
3,0.6,(Onion)
4,0.6,(Milk)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Kidney Beans, Yogurt)"
7,0.6,"(Eggs, Onion)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Eggs, Onion, Kidney Beans)"


In [25]:
data = pd.DataFrame(dataset)
data

,0,1,2,3,4,5
0,Milk,Onion,Nutmeg,Kidney Beans,Eggs,Yogurt
1,Dill,Onion,Nutmeg,Kidney Beans,Eggs,Yogurt
2,Milk,Apple,Kidney Beans,Eggs,None,None
3,Milk,Unicorn,Corn,Kidney Beans,Yogurt,None
4,Corn,Onion,Onion,Kidney Beans,Ice cream,Eggs


ECLAT

In [37]:
# we are looking for itemSETS
# we do not want to have any individual products returned
min_n_products = 2

# we want to set min support to 4 
# but we have to express it as a percentage
min_support = 4/len(dataset)

# we have no limit on the size of association rules
# so we set it to the longest transaction
max_length = max([len(x) for x in dataset])

In [38]:

# create an instance of eclat
my_eclat = ECLAT(data=data, verbose=True)

# fit the algorithm
rule_indices, rule_supports = my_eclat.fit(min_support=min_support,
                                           min_combination=min_n_products,
                                           max_combination=max_length)

100%|██████████| 12/12 [00:00<00:00, 1090.04it/s]
1it [00:00, 47.61it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

Combination 2 by 2
Combination 3 by 3
Combination 4 by 4
Combination 5 by 5
Combination 6 by 6


In [39]:
print(rule_supports)

{'Eggs & Kidney Beans': 0.8}
